In [28]:
import html
import json
import os
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [29]:
# Configuration des headers pour simuler un navigateur
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
}

In [30]:
# Fonction pour rechercher des produits
def recherche_produit(data, products_list):
    if isinstance(data, dict):
        # Si c'est un produit, l'ajouter
        if 'bestprice' in data and 'title' in data:
            produit = {
                'titre': data.get('title', 'Titre non disponible'),
                'prix_solde': data.get('bestprice'),
                'prix_avant_reduction': data.get('oldprice'),
                'note': float(data.get('rate', 0)) if data.get('rate') else None,
                'lien_image': data.get('preview', '')
            }
            products_list.append(produit)

        # Continuer la recherche dans les valeurs
        for value in data.values():
            recherche_produit(value, products_list)

    elif isinstance(data, list):
        # Parcourir chaque élément de la liste
        for item in data:
            recherche_produit(item, products_list)

In [31]:
# Fonction pour extraire le contenu d'une page web
def extrait_contenu_page(contenu):
    soup = BeautifulSoup(contenu, "html.parser")
    app_div = soup.find('div', {'id': 'app'})

    try:
        if app_div is None:
            return []
        data_page = app_div.attrs.get('data-page') if hasattr(app_div, 'attrs') else None

        if not data_page or not isinstance(data_page, str):
            return []
        
        decoded_data = html.unescape(data_page)
        page_data = json.loads(decoded_data)

        products = []
        recherche_produit(page_data, products)

        return products

    except Exception:
        return []

In [32]:
# Fonction principale pour récupérer la liste des fleurs
def get_products(total_pages: int = 2):
    base_url = "https://fleur-quebec.com/catalog/fleurs/bouquets-melanges"
    flowers_list = []

    for page in range(1, total_pages + 1):
        page_url = f"{base_url}?page={page}"
        print(f"Scraping de la page {page_url}")

        try:
            response = requests.get(page_url, headers=headers, timeout=15)
            if response.status_code != 200:
                print(f"Erreur HTTP {response.status_code} pour la page {page}")
                continue

            products_temp = extrait_contenu_page(response.text)

            if not products_temp:
                print(f"Aucun produit trouvé sur la page {page}")
                break

            flowers_list += products_temp
            print(f"{len(products_temp)} produits trouvés sur la page {page}")

            # Pause pour éviter de surcharger le serveur
            time.sleep(2)

        except Exception as e:
            print(f"Erreur lors du scraping de la page {page}: {e}")
            continue

    return flowers_list

In [33]:
# Exécution du scraping
print("Début du scraping...")

# Récupération des produits (2 pages par défaut)
produits = get_products(200)
print(f"Résultats: {len(produits)} produits extraits au total")


Début du scraping...
Scraping de la page https://fleur-quebec.com/catalog/fleurs/bouquets-melanges?page=1
12 produits trouvés sur la page 1
Scraping de la page https://fleur-quebec.com/catalog/fleurs/bouquets-melanges?page=2
12 produits trouvés sur la page 2
Résultats: 24 produits extraits au total


In [34]:
produits

[{'titre': '12 Roses Orange à Longue Tige',
  'prix_solde': 44.99,
  'prix_avant_reduction': '69.99',
  'note': 4.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/12_roses_orange__longue_tige_thumb.webp?lastmode=1751510201?lastmode=1751510201'},
 {'titre': 'Délice Rose & Rouge',
  'prix_solde': 69.99,
  'prix_avant_reduction': '109.99',
  'note': 4.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/dlice_rose__rouge_thumb.webp?lastmode=1751510201?lastmode=1751510201'},
 {'titre': 'Bouquet Chérie',
  'prix_solde': 44.99,
  'prix_avant_reduction': '74.99',
  'note': 4.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/bouquet_chrie__thumb.webp?lastmode=1751510201?lastmode=1751510201'},
 {'titre': 'Marguerites Ensoleillées',
  'prix_solde': 29.99,
  'prix_avant_reduction': '49.99',
  'note': 4.0,
  'lien_image': 'https://live.data.necs.ca/storage/products/sunshine_daisies_thumb.webp?lastmode=1751510201?lastmode=1751510201'},
 {'titre': 'Meilleur Vende

In [35]:
# Traitement des données et création du DataFrame
df = pd.DataFrame(produits) if produits else pd.DataFrame()

if not df.empty:
    print(f"✅ DataFrame créé avec succès!")
    print(f"📊 Nombre de lignes: {len(df)}")
    print(f"📋 Colonnes: {list(df.columns)}")
    print("\n🔍 Aperçu des données:")
    display(df.head())
else:
    print("❌ Aucune donnée à traiter")


✅ DataFrame créé avec succès!
📊 Nombre de lignes: 24
📋 Colonnes: ['titre', 'prix_solde', 'prix_avant_reduction', 'note', 'lien_image']

🔍 Aperçu des données:


,titre,prix_solde,prix_avant_reduction,note,lien_image
0,12 Roses Orange à Longue Tige,44.99,69.99,4.0,https://live.data.necs.ca/storage/products/12_...
1,Délice Rose & Rouge,69.99,109.99,4.0,https://live.data.necs.ca/storage/products/dli...
2,Bouquet Chérie,44.99,74.99,4.0,https://live.data.necs.ca/storage/products/bou...
3,Marguerites Ensoleillées,29.99,49.99,4.0,https://live.data.necs.ca/storage/products/sun...
4,Meilleur Vendeur,90.99,90.99,4.0,https://live.data.necs.ca/storage/products/mei...


In [36]:
# Sauvegarde des données en CSV
if not df.empty:
    # Sauvegardons le résultat dans un fichier CSV
    filename = 'resultat.csv'
    df.to_csv(filename, index=False, encoding='utf-8')

    print(f"💾 Données sauvegardées dans '{filename}'")
    print(f"📁 Chemin complet : {os.path.abspath(filename)}")
    
    # Statistiques rapides
    print(f"\n📈 Résumé des données sauvegardées:")
    print(f"   • {len(df)} produits de fleurs")
    print(f"   • Prix moyen: {df['prix_solde'].mean():.2f}$")
    print(f"   • Note moyenne: {df['note'].mean():.1f}/5")
    print(f"   • Fichier prêt pour l'analyse dans 'analyse_visualisation.ipynb'")
else:
    print("❌ Aucune donnée à sauvegarder")


💾 Données sauvegardées dans 'resultat.csv'
📁 Chemin complet : /Users/raharison/docs/TP/exam-intra-H2025/resultat.csv

📈 Résumé des données sauvegardées:
   • 24 produits de fleurs
   • Prix moyen: 48.49$
   • Note moyenne: 4.4/5
   • Fichier prêt pour l'analyse dans 'analyse_visualisation.ipynb'
